Test Full Pipeline For BMI detection using extra features extracted as well

In [1]:
# conda create -n bmi-predictor python=3.10 -y
# conda activate bmi-predictor

# # For MPS on macOS with ARM (M1/M2/M3)
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

# pip install facenet-pytorch
# pip install pandas scikit-learn matplotlib tqdm jupyter notebook
# pip install opencv-python pillow

### Data Preprocessing

In [2]:
# ===============================================
# STEP 0: Imports and Setup
# ===============================================
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, MTCNN
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
from tqdm import tqdm 

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Core count: {os.cpu_count()}')
torch.set_num_threads(os.cpu_count())  # Use all available CPU cores
DEVICE = torch.device("cpu")  # Force CPU usage for portability
print("Using device:", DEVICE)

Core count: 10
Using device: cpu


In [3]:
# Paths
IMAGE_DIR = "data/Images"
existing_image_files = os.listdir(IMAGE_DIR)

# Load CSV
data_df = pd.read_csv("data/data.csv").drop(columns=["Unnamed: 0"])
print(f"Loaded {data_df.shape[0]} rows from CSV")

# Filter out rows with missing image files
data_df = data_df[data_df["name"].isin(existing_image_files)]
print(f"Filtered to {data_df.shape[0]} rows with existing images")

del existing_image_files
data_df.head(5)

Loaded 4206 rows from CSV
Filtered to 3962 rows with existing images


,bmi,gender,is_training,name
0,34.207396,Male,1,img_0.bmp
1,26.453720,Male,1,img_1.bmp
2,34.967561,Female,1,img_2.bmp
3,22.044766,Female,1,img_3.bmp
6,25.845588,Female,1,img_6.bmp


In [4]:
features_df = pd.read_csv('data/Feature_Add.csv')
features_df.head()

,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,image,race,pred_gender,age,key,bmi,gender,is_training,name
0,237.002110,210.287898,237.008439,52.009614,120.016666,58.034473,62.008064,46.173586,42.107007,1.127036,2.307586,img_1066_face0.jpg,White,Female,20-29,img_1066,41.191406,Female,1,img_1066.bmp
1,226.035395,200.010000,220.002273,51.000000,104.076895,38.052595,62.072538,43.011626,41.048752,1.130120,2.040723,img_1585_face0.jpg,White,Male,20-29,img_1585,24.658895,Male,1,img_1585.bmp
2,241.101638,209.193690,239.075302,42.011903,87.005747,32.062439,65.000000,42.107007,43.011626,1.152528,2.070978,img_3852_face0.jpg,White,Female,20-29,img_3852,39.151259,Female,0,img_3852.bmp
3,229.490741,198.494332,229.176788,53.037722,80.056230,12.000000,54.083269,42.011903,41.012193,1.156158,1.509421,img_1857_face0.jpg,East Asian,Male,30-39,img_1857,25.845588,Male,1,img_1857.bmp
4,235.552542,201.613492,235.766834,48.041649,77.524190,27.892651,60.299254,34.014703,41.194660,1.168337,1.613687,img_4196_face0.jpg,White,Male,20-29,img_4196,36.243556,Male,0,img_4196.bmp


In [5]:
df = pd.merge(data_df, features_df, on=['gender', 'bmi', 'name', 'is_training'], how='inner')
df.drop(columns=['key', 'image', 'gender'], inplace=True)
print("Combined dataset shape:", df.shape)
df.head()

Combined dataset shape: (3712, 17)


,bmi,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,race,pred_gender,age
0,34.207396,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,44.045431,43.046487,1.374692,1.940412,White,Male,30-39
1,26.453720,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,41.012193,43.185646,1.227341,1.832808,White,Male,20-29
2,34.967561,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,42.011903,44.045431,1.240158,2.146238,White,Female,20-29
3,22.044766,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,41.000000,41.012193,1.128941,2.316792,White,Female,20-29
4,25.845588,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,48.259714,47.010637,1.218169,1.937610,White,Female,10-19


In [6]:
# Normalize target variable
bmi_mean = df['bmi'].mean()
bmi_std = df['bmi'].std()
df['bmi_norm'] = (df['bmi'] - bmi_mean) / bmi_std
df.head()

,bmi,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,race,pred_gender,age,bmi_norm
0,34.207396,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,44.045431,43.046487,1.374692,1.940412,White,Male,30-39,0.191720
1,26.453720,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,41.012193,43.185646,1.227341,1.832808,White,Male,20-29,-0.762228
2,34.967561,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,42.011903,44.045431,1.240158,2.146238,White,Female,20-29,0.285244
3,22.044766,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,41.000000,41.012193,1.128941,2.316792,White,Female,20-29,-1.304670
4,25.845588,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,48.259714,47.010637,1.218169,1.937610,White,Female,10-19,-0.837048


In [7]:
#### General information ####
print(f"Entire data shape: {df.shape}")
gender_total = df['pred_gender'].value_counts() / len(df)
gender_total = gender_total.reset_index()
gender_total.rename(columns={'count': 'percentage'}, inplace=True)
print(f"Gender Distribution Over Entire Dataset:\n{gender_total}\n")

# Split into training and testing sets
training_data_df = df[df['is_training'] == 1]
testing_data_df = df[df['is_training'] == 0]

gender_training_data_df = training_data_df['pred_gender'].value_counts() / len(training_data_df)
gender_training_data_df = gender_training_data_df.reset_index()
gender_training_data_df.rename(columns={'count': 'percentage'}, inplace=True)
print(f"Training data shape: {training_data_df.shape}")
print(f"Gender Distribution Over Training Dataset:\n{gender_training_data_df}\n")

gender_testing_data_df = testing_data_df['pred_gender'].value_counts() / len(testing_data_df)
gender_testing_data_df = gender_testing_data_df.reset_index()
gender_testing_data_df.rename(columns={'count': 'percentage'}, inplace=True)
print(f"Testing data shape: {testing_data_df.shape}")
print(f"Gender Distribution Over Testing Dataset:\n{gender_testing_data_df}\n")

del data_df, features_df, gender_total, training_data_df, testing_data_df, gender_training_data_df, gender_testing_data_df

Entire data shape: (3712, 18)
Gender Distribution Over Entire Dataset:
  pred_gender  percentage
0        Male    0.581088
1      Female    0.418912

Training data shape: (3008, 18)
Gender Distribution Over Training Dataset:
  pred_gender  percentage
0        Male    0.590093
1      Female    0.409907

Testing data shape: (704, 18)
Gender Distribution Over Testing Dataset:
  pred_gender  percentage
0        Male    0.542614
1      Female    0.457386



In [8]:
### One Hot Encode ###
race_dummies = pd.get_dummies(df['race'], prefix='race').astype(int)
gender_dummies = pd.get_dummies(df['pred_gender'], prefix='gender').astype(int)
age_dummies = pd.get_dummies(df['age'], prefix='age').astype(int)

# Fix column names immediately after get_dummies
race_dummies.columns = [col.replace(" ", "_") for col in race_dummies.columns]
gender_dummies.columns = [col.replace(" ", "_") for col in gender_dummies.columns]
age_dummies.columns = [col.replace(" ", "_") for col in age_dummies.columns]

# Combine with original numeric features
df_final = pd.concat([
    df.drop(['race', 'pred_gender', 'age'], axis=1),
    race_dummies,
    gender_dummies,
    age_dummies
], axis=1)


del df, race_dummies, gender_dummies, age_dummies
df_final.head()

,bmi,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,...,race_Latino_Hispanic,race_White,gender_Female,gender_Male,age_10-19,age_20-29,age_3-9,age_30-39,age_40-49,age_50-59
0,34.207396,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,...,0,1,0,1,0,0,0,1,0,0
1,26.453720,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,...,0,1,0,1,0,1,0,0,0,0
2,34.967561,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,...,0,1,1,0,0,1,0,0,0,0
3,22.044766,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,...,0,1,1,0,0,1,0,0,0,0
4,25.845588,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,...,0,1,1,0,1,0,0,0,0,0


In [9]:
df_final.columns

Index(['bmi', 'is_training', 'name', 'jaw_width', 'face_height',
       'cheekbone_width', 'nose_width', 'mouth_width', 'mouth_height',
       'eye_distance', 'left_eye_width', 'right_eye_width',
       'face_width_to_height', 'mouth_to_nose_ratio', 'bmi_norm', 'race_Black',
       'race_East_Asian', 'race_Latino_Hispanic', 'race_White',
       'gender_Female', 'gender_Male', 'age_10-19', 'age_20-29', 'age_3-9',
       'age_30-39', 'age_40-49', 'age_50-59'],
      dtype='object')

In [10]:
extra_feature_cols = [col for col in df_final.columns if col not in ['name', 'bmi', 'bmi_norm' 'is_training']]
print(df_final[extra_feature_cols].dtypes[df_final[extra_feature_cols].dtypes == "object"])

Series([], dtype: object)


### Face Detection & Alignment using MTCNN

In [11]:
# ===============================================
# STEP X: Precompute and Cache Face Embeddings
# ===============================================
from facenet_pytorch import InceptionResnetV1, MTCNN
from torchvision import transforms
import torch
from PIL import Image
import os
from tqdm import tqdm
import numpy as np

IMAGE_DIR = "data/Images"
EMBEDDING_CACHE_DIR = "data/embeddings"
os.makedirs(EMBEDDING_CACHE_DIR, exist_ok=True)

# mtcnn = MTCNN(image_size=160, margin=20, post_process=True, device=device)
mtcnn = MTCNN(image_size=224, margin=20, post_process=True, device=device)
embedder = InceptionResnetV1(pretrained="vggface2").eval().to(device)

def extract_face_embedding(image_path):
    img = Image.open(image_path).convert("RGB")
    face = mtcnn(img)
    if face is None:
        # return None
        return torch.zeros(512).numpy()
    face = face.unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = embedder(face).squeeze(0).cpu().numpy()
    return embedding

generate_emb = input("Generate embeddings?: Y or N").lower().strip() or "n"

if generate_emb == 'y':
    # Generate embeddings
    for idx, row in tqdm(df_final.iterrows(), total=len(df_final), desc="Caching embeddings"):
        image_name = row["name"]
        image_path = os.path.join(IMAGE_DIR, image_name)
        save_path = os.path.join(EMBEDDING_CACHE_DIR, image_name.replace(".bmp", ".npy"))

        # if not os.path.exists(save_path):
        try:
            embedding = extract_face_embedding(image_path)
            if embedding is not None:
                np.save(save_path, embedding)
        except Exception as e:
            print(f"Failed {image_name}: {e}")
else: 
    print("Will use saved embeddings from prior process")

Will use saved embeddings from prior process


In [12]:
# Remove images we were not able to create embeddings 
from pathlib import Path
df_final = df_final[df_final["name"].apply(lambda x: Path("data/embeddings") / x.replace(".bmp", ".npy")).apply(Path.exists)].reset_index(drop=True)
print(df_final.shape)

(3712, 27)


In [28]:
from torchvision import transforms as T

image_transforms = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.1, contrast=0.1),
    T.ToTensor()
])

class BMIDataset(Dataset):
    def __init__(self, dataframe, image_dir, embedding_dir="data/embeddings", transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.embedding_dir = embedding_dir
        self.transform = transform
        self.device = device

        self.mtcnn = MTCNN(image_size=224, margin=20, post_process=True, device=self.device)
        self.embedder = InceptionResnetV1(pretrained='vggface2').eval().to(self.device)


        # Identify tabular feature columns
        self.extra_feature_cols = [
            col for col in dataframe.columns
            if col not in ['name', 'bmi', 'bmi_norm', 'is_training']
        ]
        self.scaler = StandardScaler()
        self.df[self.extra_feature_cols] = self.scaler.fit_transform(self.df[self.extra_feature_cols])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row['name'])

        img = Image.open(image_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        # Face alignment
        face = self.mtcnn(img)
        if face is None:
            face_embedding = torch.zeros(512)
        else:
            face = face.unsqueeze(0).to(self.device)
            with torch.no_grad():
                face_embedding = self.embedder(face).squeeze(0).cpu()

        # extra_features = torch.tensor(row[self.extra_feature_cols].values, dtype=torch.float32)
        extra_features = torch.tensor(row[self.extra_feature_cols].values.astype(np.float32), dtype=torch.float32)
        bmi = torch.tensor(row['bmi_norm'], dtype=torch.float32)

        return face_embedding, extra_features, bmi

In [29]:
# ===============================================
# STEP 5: Model Definition (Multitask)
# ===============================================
class BMIMultitaskModel(nn.Module):
    def __init__(self, extra_feature_dim):
        super().__init__()
        self.backbone = None  # no longer needed with cached embeddings

        self.shared = nn.Sequential(
            nn.Linear(512 + extra_feature_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
        )

        self.bmi_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        self.age_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        self.gender_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # or 2 if using softmax
        )

    def forward(self, face_embed, extra_features):
        x = torch.cat([face_embed, extra_features], dim=1)
        x = self.shared(x)

        bmi = self.bmi_head(x).squeeze(1)
        age = self.age_head(x).squeeze(1)
        gender = self.gender_head(x).squeeze(1)  # sigmoid if binary

        return bmi, age, gender

In [30]:
# ===============================================
# STEP 6: Prepare Data Loaders
# ===============================================

# Obtain values to normalize target variable

X_train = df_final[df_final['is_training'] == 1]
test_df = df_final[df_final['is_training'] == 0]

# Create validation set
validation_set_percent = 0.3

train_df, validation_df = train_test_split(X_train, test_size=0.3, random_state=42)

print(f"Training set shape: {train_df.shape}")
print(f"Validation set shape: {validation_df.shape}")
print(f"Number of Males in training set shape: {train_df['gender_Male'].sum()}; {train_df['gender_Male'].sum()/len(train_df) * 100:.2f}%")
print(f"Number of Females in training set shape: {train_df['gender_Female'].sum()}; {train_df['gender_Female'].sum()/len(train_df) * 100:.2f}%")
print(f"Number of Males in validation set shape: {validation_df['gender_Male'].sum()}; {validation_df['gender_Male'].sum()/len(validation_df) * 100:.2f}%")
print(f"Number of Females in validation set shape: {validation_df['gender_Female'].sum()}; {validation_df['gender_Female'].sum()/len(validation_df) * 100:.2f}%")

train_dataset = BMIDataset(train_df, image_dir=IMAGE_DIR)
val_dataset = BMIDataset(validation_df, image_dir=IMAGE_DIR)
test_dataset = BMIDataset(test_df, image_dir=IMAGE_DIR)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

Training set shape: (2105, 27)
Validation set shape: (903, 27)
Number of Males in training set shape: 1241; 58.95%
Number of Females in training set shape: 864; 41.05%
Number of Males in validation set shape: 534; 59.14%
Number of Females in validation set shape: 369; 40.86%


In [31]:
# Configure paths for best model
MODEL_PATH = "saved_models/best_bmi_prediction_model.pth"
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

In [32]:
# ===============================================
# STEP 7: Training Loop
# ===============================================

def evaluate(model, criterion, dataloader):
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for face, extra, bmi in tqdm(dataloader, desc="Evaluating", leave=False):
            face = face.to(DEVICE, non_blocking=True)
            extra = extra.to(DEVICE, non_blocking=True)
            bmi = bmi.to(DEVICE, non_blocking=True)

            pred_bmi, _, _ = model(face, extra)
            loss = criterion(pred_bmi, bmi)
            val_loss += loss.item()
            all_preds.append(pred_bmi.cpu().numpy())
            all_labels.append(bmi.cpu().numpy())
    
    avg_val_loss = val_loss / len(dataloader)
    
    preds = np.concatenate(all_preds)
    labels = np.concatenate(all_labels)
    
    # Reverse transformation from normalized bmi values
    preds = preds * bmi_std + bmi_mean
    labels = labels * bmi_std + bmi_mean
    
    # Evaluate performance
    mae = mean_absolute_error(labels, preds)
    r2 = r2_score(labels, preds)
    pearson_corr, _ = pearsonr(labels, preds)

    print(f"Evaluation: MAE={mae:.2f}, R²={r2:.3f}, Pearson r={pearson_corr:.3f}")
    return avg_val_loss, mae, r2, pearson_corr

def train_epoch(model, optimizer, criterion, dataloader):
    model.train()
    total_loss = 0
    for face, extra, bmi in tqdm(dataloader, desc="Training", leave=False):
        # face, extra, bmi = face.to(DEVICE), extra.to(DEVICE), bmi.to(DEVICE)
        face = face.to(DEVICE, non_blocking=True)
        extra = extra.to(DEVICE, non_blocking=True)
        bmi = bmi.to(DEVICE, non_blocking=True)
        
        optimizer.zero_grad()
        pred_bmi, _, _ = model(face, extra)  # Only use BMI for loss
        loss = criterion(pred_bmi, bmi)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

def train_model(model, optimizer, criterion, epochs=150):
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        loss = train_epoch(model, optimizer, criterion, train_loader)
        avg_val_loss, val_mae, r2_value, pearson_corr = evaluate(model, criterion, validation_loader)
        print(f"Epoch [{epoch+1}/{epochs}]: Train Loss={loss:.4f}, Val Loss={avg_val_loss:.4f}, Val MAE={val_mae:.3f}, Val R^2={r2_value:.3f}, Val Pearson Coefficient={pearson_corr:.3f}")
        
        # Save best model
        if avg_val_loss < best_val_loss:
            print(f"Prior best: {best_val_loss}")
            best_val_loss = avg_val_loss
            print(f"Current best: {best_val_loss}")
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"Best model saved to: {MODEL_PATH}")

def load_best_model(extra_feature_dim):
    model = BMIMultitaskModel(extra_feature_dim=extra_feature_dim).to(DEVICE)
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint)
    return model

In [33]:
# Train model
is_train_model = input("Train new model? Y or N\n Default: Use best saved model").lower().strip() or "n"

model = BMIMultitaskModel(extra_feature_dim=len(train_dataset.extra_feature_cols)).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

if is_train_model == 'y':
    print("Beginning training")
    train_model(model, optimizer, criterion)
    print("Training Complete")
else:
    print("Loading saved best model")
# Get extra features dimension
extra_feature_dim = len(test_dataset.extra_feature_cols)
best_model = load_best_model(extra_feature_dim)

Beginning training


Evaluation: MAE=6.35, R²=0.047, Pearson r=0.490
Epoch [1/150]: Train Loss=0.9173, Val Loss=0.9593, Val MAE=6.349, Val R^2=0.047, Val Pearson Coefficient=0.490
Prior best: inf
Current best: 0.9593148745339493
Best model saved to: saved_models/best_bmi_prediction_model.pth


Evaluation: MAE=6.05, R²=0.132, Pearson r=0.511
Epoch [2/150]: Train Loss=0.8570, Val Loss=0.8710, Val MAE=6.046, Val R^2=0.132, Val Pearson Coefficient=0.511
Prior best: 0.9593148745339493
Current best: 0.8709916143581785
Best model saved to: saved_models/best_bmi_prediction_model.pth


Evaluation: MAE=5.67, R²=0.234, Pearson r=0.536
Epoch [3/150]: Train Loss=0.7674, Val Loss=0.7734, Val MAE=5.668, Val R^2=0.234, Val Pearson Coefficient=0.536
Prior best: 0.8709916143581785
Current best: 0.7733697809022049
Best model saved to: saved_models/best_bmi_prediction_model.pth


Evaluation: MAE=5.33, R²=0.299, Pearson r=0.560
Epoch [4/150]: Train Loss=0.6879, Val Loss=0.7051, Val MAE=5.328, Val R^2=0.299, Val Pearson Coefficient=0.560
Prior best: 0.7733697809022049
Current best: 0.7051490298632918
Best model saved to: saved_models/best_bmi_prediction_model.pth


...

Evaluation: MAE=4.92, R²=0.384, Pearson r=0.621
Epoch [7/150]: Train Loss=0.5839, Val Loss=0.6229, Val MAE=4.924, Val R^2=0.384, Val Pearson Coefficient=0.621
Prior best: 0.6424722589295486
Current best: 0.6229321771654589
Best model saved to: saved_models/best_bmi_prediction_model.pth


Evaluation: MAE=4.85, R²=0.395, Pearson r=0.631
Epoch [8/150]: Train Loss=0.5656, Val Loss=0.6133, Val MAE=4.849, Val R^2=0.395, Val Pearson Coefficient=0.631
Prior best: 0.6229321771654589
Current best: 0.6132939098210171
Best model saved to: saved_models/best_bmi_prediction_model.pth


Evaluation: MAE=4.81, R²=0.410, Pearson r=0.643
Epoch [9/150]: Train Loss=0.5506, Val Loss=0.6063, Val MAE=4.813, Val R^2=0.410, Val Pearson Coefficient=0.643
Prior best: 0.6132939098210171
Current best: 0.6062649745365669
Best model saved to: saved_models/best_bmi_prediction_model.pth


Evaluation: MAE=4.77, R²=0.416, Pearson r=0.646
Epoch [10/150]: Train Loss=0.5397, Val Loss=0.5935, Val MAE=4.773, Val R^2=0.416, Val Pearson Coefficient=0.646
Prior best: 0.6062649745365669
Current best: 0.5935370243828872
Best model saved to: saved_models/best_bmi_prediction_model.pth


KeyboardInterrupt: 

In [ ]:
# Results from testing set 
model = BMIMultitaskModel(extra_feature_dim=len(train_dataset.extra_feature_cols)).to(DEVICE)
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint)
model.eval()
_, test_mae, test_r2, test_pearson = evaluate(model, criterion, test_loader)
print(f"\nTesting Set MAE={test_mae:.3f}, R²={test_r2:.3f}, Pearson Coefficient={test_pearson:.3f}")

In [41]:
# import torch
# import numpy as np
# from PIL import Image
# import os

# ===============================================
# Load a single image and predict BMI
# ===============================================
def predict_outputs(image_name, model, extra_feature_df, embedding_dir="data/embeddings"):
    model.eval()
    
    embedding_path = os.path.join(embedding_dir, image_name.replace(".bmp", ".npy"))
    if not os.path.exists(embedding_path):
        raise FileNotFoundError(f"Embedding not found for: {image_name}")

    face_tensor = torch.tensor(np.load(embedding_path), dtype=torch.float32).unsqueeze(0)

    row = extra_feature_df[extra_feature_df["name"] == image_name]
    if row.empty:
        raise ValueError(f"No extra feature data for {image_name}")

    extra_cols = train_dataset.extra_feature_cols
    extra_features = torch.tensor(row[extra_cols].values.astype(np.float32), dtype=torch.float32)

    with torch.no_grad():
        pred_bmi, pred_age, pred_gender_logits = model(face_tensor, extra_features)
        pred_gender = "Male" if torch.sigmoid(pred_gender_logits).item() > 0.5 else "Female"

    pred_bmi = pred_bmi.item() * bmi_std + bmi_mean
    
    return pred_bmi, pred_age.item(), pred_gender

In [ ]:
test_image = "img_2.bmp" 

pred_bmi, _, pred_gender = predict_outputs(test_image, model, df_final)
actual_bmi = df_final[df_final['name'] == test_image]['bmi'].iloc[0]
print(f"Predicted BMI: {pred_bmi:.2f}, Gender: {pred_gender}")
print(f"Actual BMI for {test_image}: {actual_bmi:.2f}")